### **Cos'è LangChain?**
> LangChain è un framework per lo sviluppo di applicazioni che sfruttano modelli di linguaggio.

**TLDR**: LangChain rende più semplici le parti complicate di utilizzo e costruzione di modelli di intelligenza artificiale. Ciò avviene in due modi:

1. **Integration** - Importa dati esterni, come i filed, altre applicazioni o dati da API, ai tuoi modelli di linguaggio.
2. **Agentività** - Consente ai modelli di linguaggio di interagire con l'ambiente prendendo decisioni e utilizzando tools esterni. Utilizza i modelli di linguaggio per aiutarti a decidere quale azione intraprendere successivamente.

### **Perché LangChain?**
1. **Components** - LangChain facilita la sostituzione e l'astrazione dei componenti necessari per lavorare con modelli di linguaggio.

2. **Chain personalizzate** - LangChain fornisce supporto integrato per l'utilizzo e la personalizzazione di 'chain' - una serie di azioni concatenate.

3. **Velocità 🚢** - Il team è incredibilmente veloce nel rilascio. Sarai sempre aggiornato sulle ultime funzionalità dei modelli di linguaggio.

4. **Community 👥** - Supporto tramite Discord e comunità, incontri, hackathon, ecc.


In [9]:
# Load .env file
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="../.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_api_key=OPENAI_API_KEY

# Componenti LangChain

## Schema di input
### **Text**
Il modo di interagire con i modelli utilizzando il linguaggio naturale


In [10]:
# Possiamo lavorare con testi semplici
my_text = "Che giorno viene dopo Venerdì?"

### **Chat Messages**
Simili al testo, ma vengo associati con un tipo di messaggio (System, Human, AI)

* **System** - Contesto di sfondo utile che indica all'AI cosa fare
* **Human** - Messaggi che intendono rappresentare l'utente
* **AI** - Messaggi che mostrano ciò a cui l'AI ha risposto

Per ulteriori informazioni, consulta la [documentazione](https://platform.openai.com/docs/guides/chat/introduction) di OpenAI.


In [11]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

history = [
    SystemMessage(content="You are an helpful chatbot that helps a user to remember the days of the week"),
    HumanMessage(content=my_text),
]

### **Documents**
Un oggetto che contiene un stringa di testo e dei metadati (ulteriori informazioni su quel testo)

In [12]:
from langchain.schema import Document
Document(page_content="Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
         metadata={
             'id' : 42,
             'source' : "https://www.loremipsum.com",
             'create_time' : "2021-01-01 12:00:00",
         })

Document(page_content='Lorem ipsum dolor sit amet, consectetur adipiscing elit.', metadata={'id': 42, 'source': 'https://www.loremipsum.com', 'create_time': '2021-01-01 12:00:00'})

## Models - The interface to the AI brains

###  **Language Model**
Un modello di linguaggio è un modello che prende in input una stringa di testo e restituisce una stringa di testo.

In [13]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)

In [14]:
llm(my_text)

'\nSabato'

### **Chat Model**
Un modello di chat è un modello che prende in input una serie di messaggi e restituisce un messaggio di output

In [15]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=1, openai_api_key=openai_api_key)

In [16]:
chat(history)

AIMessage(content='Dopo Venerdì, viene Sabato.', additional_kwargs={}, example=False)

### **Text Embedding Model**
Un modello di embedding di testo è un modello che prende in input una stringa di testo e restituisce un vettore (una serie di numeri che rappresentano il significato semantico del testo).
Principalmente utilizzato quando si confrontano due stringe di testo insieme.

In [17]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [18]:
text = "Sono un appassionato di tecnologia e mi piace leggere libri di fantascienza."

In [19]:
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 1536
Here's a sample: [-0.005110122263431549, -0.0038479098584502935, 0.00033489224733784795, -0.032817523926496506, -0.026102064177393913]...


Proviamo a vedere la distanza del testo tra altre due stringhe di testo:

In [ ]:
! pip install scipy -q

In [35]:
from scipy import spatial
text_embedding_1 = embeddings.embed_query('Mi piace leggere libri di fantascienza.')
text_embedding_2 = embeddings.embed_query('Non riesco veramente a capire come funziona la tecnologia.')

result_1 = 1 - spatial.distance.cosine(text_embedding, text_embedding_1)
result_2 = 1 - spatial.distance.cosine(text_embedding, text_embedding_2)

print (f"Similarity between text and text_1: {result_1}")
print (f"Similarity between text and text_2: {result_2}")

Similarity between text and text_1: 0.9377983460525731
Similarity between text and text_2: 0.8337677010188276


## Prompts - Testo utilizzato per dare istruzioni al tuo modello

### **Prompt**
Cosa dirai al tuo modello.

In [22]:
from langchain.llms import OpenAI

llm = OpenAI( openai_api_key=openai_api_key)

prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

llm(prompt)

'\nThe statement is incorrect; tomorrow is Tuesday, not Wednesday.'

### **Prompt Template**
Un oggetto che aiuta a creare prompt basati su una combinazione di input utente, altre informazioni non statiche e una stringa di modello fissa.

Come se fosse una [f-string](https://realpython.com/python-f-strings/) in python ma per i prompt.

In [23]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
LLM Output: Visit the Colosseum, the Vatican, and the Trevi fountain.


## Chains 

Cobinazione di diverse chiamate LLM e possibili azioni automatiche.

Ex: Summary #1, Summary #2, Summary #3 > Final Summary

Guarda [questo video](https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=2s) che spiega i diversi tipi di summarization chains.

Esistono diverse [tipi di chains](https://python.langchain.com/en/latest/modules/chains/how_to_guides.html), cerca di vedere quali sono le migliori per il tuo caso d'uso. 

Noi ne useremo due:

### 1. Simple Sequential Chains

Chains semplici in cui l'output di un LLM viene utilizzato come input in un altro. Ottimo per suddividere i compiti (e mantenere l' LLM focalizzato).

In [24]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [25]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [26]:
# Proviamo la prima chain

location_chain.run("Rome")

'A classic dish from Rome is Carbonara, which is a pasta dish made with egg, bacon, and cheese.'

In [27]:
# Possibilità di fare batchin in input
input_list = [
    {"user_location": "Paris"},
    {"user_location": "Berlin"},
    {"user_location": "Lisbon"}
]

location_chain.apply(input_list)

[{'text': 'A classic dish from Paris would be Coq au Vin, a classic French dish of chicken braised in red wine with mushrooms, onions, and bacon.'},
 {'text': 'A classic dish from Berlin is the Berliner Eisbein, which is slow-cooked, pickled pork knuckle served with mashed potatoes and sauerkraut.'},
 {'text': 'A classic dish from Lisbon is the famous "Bacalhau à Brás," a dish of shredded salted cod, potatoes, onions, black olives, and scrambled eggs.'}]

In [28]:
# Seconda chain

template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [29]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [30]:
review = overall_chain.run("Rome")



> Entering new  chain...
Spaghetti alla Carbonara, a classic Roman dish with spaghetti, bacon, eggs, and Parmesan cheese.
- Boil 4oz of spaghetti for 12 minutes. 
- Sauté 8oz of bacon in a frying pan until crispy.
- Beat two eggs in a bowl with a pinch of salt and freshly ground pepper. 
- Reserve a cup of the boiling water and then drain the spaghetti. 
- Place the spaghetti into the pan with the bacon. 
- Slowly add the egg mixture and the Parmesan cheese, stirring continuously. 
- Add the reserved boiling water if needed to obtain your desired consistency. 
- Serve with freshly ground pepper and extra Parmesan cheese. Enjoy!

> Finished chain.


### 2. Summarization Chain
Chain già implementata che summarizza facilmente lunghi documenti e ne ottiene un riassunto. Guarda [questo video](https://www.youtube.com/watch?v=f9_BWhCI4Zo) per altri tipi di chain oltre alla map-reduce.

In [ ]:
! pip install tiktoken -q

In [32]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('./very_long_document.txt')
documents = loader.load()

# Prepara lo splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Dividi il documento in chunks
texts = text_splitter.split_documents(documents)

# Ci sta un po' di complessità in questa riga. Ti consiglio di guardare il video sopra per maggiori dettagli
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
res = chain.run(texts)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"Giuseppe Lippi
Le origini della Fondazione
Come è nato il più famoso ciclo fantascientifico
di tutti i tempi
Apparso sul n. 1203 di Urania (18 aprile 1993)
È il mattino del 1° agosto 1941; a New York, nella Settima Avenue, un giovanotto
di belle speranze sale i gradini del palazzo Street & Smith, la vecchia casa editrice
specializzata in pulp magazines che pubblica Astounding Science Fiction (ex
Astounding Stories). Il giovanotto ha un appuntamento col signor Campbell, sì, John
W. Campbell jr., conferma all’usciere mentre attende impaziente il pass.
Attraversa quindi alcuni corridoi, e finalmente (dopo la visione di una magica"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"stanza in cui sono accumulati, in tanti pacchetti, gli Astounding del mese dopo!),
accede all’ufficio del signor Campbell. Il giovanotto è Isaac Asimov


> Finished chain.


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"
This story begins on August 1st, 1941, when a young man with ambition arrives at the Street & Smith publishing house. After settling in he has a meeting with John W. Campbell Jr., the editor of the 'Astounding Science Fiction' magazine. Through this meeting, he creates the most famous science fiction series of all time.


Isaac Asimov, a promising new author who has already sold five stories, pays a visit to John Campbell's office at Astounding Science Fiction. During their story conferences, Asimov forgets the world outside and focuses solely on Astounding, Campbell, and the stories they are discussing. On August 1st, 1941, Asimov does not think about the looming threat of Hitler.

 Feeling desperate due to not having a case to present to his boss, Asimov randomly opens a book to find a page from Gilbert & Sullivan's Iolanthe in which the Queen of the Fairies falls at t

In [33]:
import pprint
pprint.pprint(res)

(' Isaac Asimov\'s iconic "Foundation" series begins when the author visits '
 'the Street & Smith publishing house and pitches a concept of a galactic '
 'empire based on the Roman Empire to John Campbell, the editor of Astounding '
 'Science Fiction. Asimov successfully sells several stories and chapters in '
 'the series, such as The Big and the Little and Now You See It, until finally '
 'it is published as a complete trilogies series. Award-winning and the most '
 'popular of its genre in science fiction, Foundation has been celebrated for '
 'its complex political, economic, diplomactic, and scientific facets. It '
 'showcases components such as faster-than-light speed, intergalactic '
 'civilizations, advanced Psicostoria science, and threats of a being '
 'nicknamed the Mule.')
